# Assignment 1

## Kun Huang,Yizhou Jin,Jui-Chang Ting



## Problem 1

First, we import packages.

In [1]:
import urllib.request
from lxml import etree
import numpy as np
import pandas as pd
# the fowllowing support for uploading files to google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive


The function  takes an input `year`,which stands for the period, sends requests to US Treasury website, retrieves the response, and returns the yield rates in a 2-dimensional ndarray with `dtype = np.float32`. Moreover, it returns the corresponding date which will be used in problem 2 as index. 

In [5]:
def yield_rate (year):
    period = ["BC_1MONTH", "BC_3MONTH", "BC_6MONTH", "BC_1YEAR", "BC_2YEAR", "BC_3YEAR", "BC_5YEAR",
          "BC_7YEAR", "BC_10YEAR", "BC_20YEAR", "BC_30YEAR"]  
    xml = urllib.request.urlopen(
        'http://data.treasury.gov/feed.svc/DailyTreasuryYieldCurveRateData?$filter=year(NEW_DATE)%20eq%20'+str(year)
    ).read()  
    tree = etree.fromstring(xml)
    tree.xpath("//*")
    row=len(tree.findall(".//{http://schemas.microsoft.com/ado/2007/08/dataservices}BC_1MONTH"))
    col=len(period)
    yields = np.empty((row,0)).tolist()    
    for i in range(row):
        for j in range(col):
            tbill =tree.findall(".//{http://schemas.microsoft.com/ado/2007/08/dataservices}"+period[j])
            yields[i].append(tbill[i].text)  
    date =tree.findall(".//{http://schemas.microsoft.com/ado/2007/08/dataservices}NEW_DATE")
    DATE = []
    for i in range(row):
        DATE.append( date[i].text[:10])
    return np.array(yields,dtype = np.float32), DATE 

For example, we want the yields rate of 2017.

In [9]:
yields, date = yield_rate(2017)

In [11]:
yields[:5]

array([[0.52, 0.53, 0.65, 0.89, 1.22, 1.5 , 1.94, 2.26, 2.45, 2.78, 3.04],
       [0.49, 0.53, 0.63, 0.87, 1.24, 1.5 , 1.94, 2.26, 2.46, 2.78, 3.05],
       [0.51, 0.52, 0.62, 0.83, 1.17, 1.43, 1.86, 2.18, 2.37, 2.69, 2.96],
       [0.5 , 0.53, 0.61, 0.85, 1.22, 1.5 , 1.92, 2.23, 2.42, 2.73, 3.  ],
       [0.5 , 0.5 , 0.6 , 0.82, 1.21, 1.47, 1.89, 2.18, 2.38, 2.69, 2.97]],
      dtype=float32)

In [13]:
yields.shape

(251, 11)

## Problem 2

We takes the yields rate from 2015 to 2019. We first create a temporary list to store DataFrame, then use `pandas.concat` to merge those DataFrame storing yields in different years.

In [16]:
temp = []
col_name = ["1 Mo", "3 Mo", "6 Mo", "1 Yr", 
            "2 Yr","3 Yr", "5 Yr", "7 Yr", "10 Yr", "20 Yr", "30 Yr"]  
for year in range(2015,2020):
    yields,date = yield_rate(year)
    temp.append(pd.DataFrame(yields,index = date,columns = col_name))
    

We use `pydrive` to upload the data. Details can be found in the [offical document](https://pythonhosted.org/PyDrive/quickstart.html) 

In [14]:
gauth = GoogleAuth()
gauth.LocalWebserverAuth() 
# Creates local webserver and auto handles authentication.

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=563272299424-1d6hdqhvm3h1npjine76k1oc54oarqhs.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


Then we upload the DataFrame `whole_sheet`.

In [20]:
drive = GoogleDrive(gauth)
whole_sheet = pd.concat(temp)
writer = pd.ExcelWriter("test.xlsx")
whole_sheet.to_excel(writer,'sheet1',float_format='%.2f') 
uploaded = drive.CreateFile({'title': 'test.xlsx'})
uploaded.SetContentFile("test.xlsx")
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1bddsMHL5OoOZooBqFCfGFVfC2qErs0gn
